# Credit Risk Resampling Techniques

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the CSV into DataFrame

In [7]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [8]:
# Fitting and encoding the columns with the LabelEncoder
le = LabelEncoder()

# Encoding homeowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [11]:
# Create our features
X= df.drop(columns="loan_status")

# Create our target
y=df["loan_status"]
X

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,1,52800,0.431818,5,1,22800
1,8400.0,6.692,1,43600,0.311927,3,0,13600
2,9000.0,6.963,2,46100,0.349241,3,0,16100
3,10700.0,7.664,1,52700,0.430740,5,1,22700
4,10800.0,7.698,0,53000,0.433962,5,1,23000
5,10100.0,7.438,0,50600,0.407115,4,1,20600
6,10300.0,7.490,0,51100,0.412916,4,1,21100
7,8800.0,6.857,0,45100,0.334812,3,0,15100
8,9300.0,7.096,1,47400,0.367089,3,0,17400
9,9700.0,7.248,2,48800,0.385246,4,0,18800


In [12]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [15]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [16]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [17]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [18]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [19]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [26]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

# View the count of target classes with Counter

rando_samp = RandomOverSampler(random_state=1)
X_resampled_rando, y_resampled_rando = rando_samp.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_rando)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [27]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model_rando = LogisticRegression(solver='lbfgs', random_state=1)
model_rando.fit(X_resampled_rando, y_resampled_rando)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred_rando = model_rando.predict(X_test_scaled)

bas_rando=balanced_accuracy_score(y_test, y_pred_rando)
print(bas_rando)

0.9946414201183431


In [31]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm_rando = confusion_matrix(y_test, y_pred_rando)
cm_df_rando = pd.DataFrame(
    cm_rando, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_rando

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [32]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_rando))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [33]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

# View the count of target classes with Counter

X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
from collections import Counter

Counter(y_resampled_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [35]:
# Train the Logistic Regression model using the resampled data
mod_smote = LogisticRegression(solver='lbfgs', random_state=1)
mod_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [37]:
# Calculated the balanced accuracy score
y_predsmote = mod_smote.predict(X_test_scaled)
balance_smote=balanced_accuracy_score(y_test, y_predsmote)
print(balance_smote)

0.9946414201183431


In [39]:
# Display the confusion matrix
confusion_smote = confusion_matrix(y_test, y_predsmote)
confusion_df_smote = pd.DataFrame(
    confusion_smote, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
confusion_df_smote

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [47]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predsmote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [49]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

# View the count of target classes with Counter
centroids = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = centroids.fit_resample(X_train_scaled, y_train)

from collections import Counter

Counter(y_resampled_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [50]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_centroids = LogisticRegression(solver='lbfgs', random_state=1)
model_centroids.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [53]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_centroids = model_centroids.predict(X_test_scaled)
bas_centroid=balanced_accuracy_score(y_test, y_centroids)
print(bas_centroid)

0.9932813049736127


In [55]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_centroid = confusion_matrix(y_test, y_centroids)
cm_df_centroid = pd.DataFrame(
    cm_centroid, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_centroid

,Predicted High Risk,Predicted Low Risk
Actual High Risk,620,5
Actual Low Risk,102,18657


In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_centroids))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [58]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

# View the count of target classes with Counter
smotenn = SMOTEENN(random_state=0)
X_resampled_se, y_resampled_se= smotenn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_se)

Counter({'high_risk': 55629, 'low_risk': 55913})

In [59]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_resampled= LogisticRegression(solver='lbfgs', random_state=1)
model_resampled.fit(X_resampled_se, y_resampled_se)

LogisticRegression(random_state=1)

In [61]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_se = model_resampled.predict(X_test_scaled)
bas_resample=balanced_accuracy_score(y_test, y_pred_se)
print(bas_resample)

0.9946414201183431


In [63]:
# Display the confusion matrix
confusion_se = confusion_matrix(y_test, y_pred_se)
confusion_df_se = pd.DataFrame(
    confusion_se, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
confusion_df_se

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [64]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_se))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


In [67]:
#1) All of the models had the same balanced accuracy score. 
#2) All of the models had the same best recall score which is better than the original data.
#3) All had similar geometric mean scores. 